# Step 1 Download Dataset to be trained

In [ ]:
from roboflow import Roboflow
#load api key from environment variable
import os
from dotenv import load_dotenv
load_dotenv()

# Load the API key from the environment variable
api_key = os.getenv("ROBOFLOW_API_KEY")
if api_key is None:
    raise ValueError("API key not found. Please set the ROBOFLOW_API_KEY environment variable.")


# Initialize Roboflow (you'll need an API key)
rf = Roboflow(api_key=api_key)

# Download the dataset
project = rf.workspace("it-3pwlf").project("glasses-detection-cqd8m")
dataset = project.version(4).download("yolov11")

loading Roboflow workspace...
loading Roboflow project...
Exporting format yolov12 in progress : 85.0%
Version export complete for yolov12 format



Extracting Dataset Version Zip to glasses-detection-4 in yolov12:: 100%|██████████| 2460/2460 [00:00<00:00, 2515.47it/s]


In [11]:
#https://universe.roboflow.com/pex-5-ylpua/pex5-gxq3t/dataset/4

# Download the dataset in YOLOv11 format
from roboflow import Roboflow
rf = Roboflow(api_key="J7yWQBdTflYMTZ2dKX64")
project = rf.workspace("noriel-vlmtq").project("glassesdetection-lwja6")
version = project.version(2)
dataset = version.download("yolov11")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to GlassesDetection-2 in yolov11:: 100%|██████████| 5888/5888 [00:02<00:00, 1998.85it/s]


In [12]:
pip show ultralytics

Name: ultralytics
Version: 8.3.29
Summary: Ultralytics YOLO 🚀 for SOTA object detection, multi-object tracking, instance segmentation, pose estimation and image classification.
Home-page: https://ultralytics.com
Author: 
Author-email: Glenn Jocher <glenn.jocher@ultralytics.com>, Jing Qiu <jing.qiu@ultralytics.com>
License: AGPL-3.0
Location: c:\users\amirah\anaconda3\envs\yolo\lib\site-packages
Requires: matplotlib, numpy, opencv-python, pandas, pillow, psutil, py-cpuinfo, pyyaml, requests, scipy, seaborn, torch, torchvision, tqdm, ultralytics-thop
Required-by: 
Note: you may need to restart the kernel to use updated packages.


# Check for Cuda availability

In [1]:
import torch
print(torch.cuda.is_available())  # Should return True if a GPU is available
print(torch.cuda.device_count())  # Number of GPUs available
print(torch.cuda.get_device_name(0))  # Name of the GPU (if available)

True
1
NVIDIA GeForce RTX 4070 SUPER


# Step 2 : Training Yolov11 on the Glasses dataset

In [1]:
from ultralytics import settings

# Update a setting
settings.update({"mlflow": True})

# Reset settings to default values
settings.reset()


In [3]:
import re
from ultralytics import YOLO
import mlflow
import os
import yaml
from mlflow.tracking import MlflowClient
current_dir = r"C:\Users\Amirah\OneDrive\999 PERSONAL\MT"

def load_config(config_path):
    """Load configuration from YAML file"""
    current_dir = r"C:\Users\Amirah\OneDrive\999 PERSONAL\MT"

    config_path = os.path.join(current_dir, config_path)
    with open(config_path, 'r') as file:
        return yaml.safe_load(file)
    
def on_train_start(trainer):
    print('in the on_train_start')

def on_fit_epoch_end(trainer):
    print('in the on_fit_epoch_end')
    metrics_dict = {f"{re.sub('[()]', '', k)}": float(v) for k, v in trainer.metrics.items()}
    mlflow.log_metrics(metrics=metrics_dict, step=trainer.epoch)

def train_model():
    # Get the current working directory
    
    
    mlflow.set_tracking_uri("http://localhost:5000")  # Update if your MLflow server is elsewhere
    mlflow.set_experiment("Glasses Detection1")
    # End any active run before starting a new one
    if mlflow.active_run():
        mlflow.end_run()
    app_config = load_config('configs/app_config.yaml')
    mlflow_config = load_config('configs/mlflow.yaml')
    # Start MLflow run
    with mlflow.start_run():
        model = YOLO(app_config['model']['path'])
        model.add_callback("on_train_start", on_train_start)
        model.add_callback("on_fit_epoch_end", on_fit_epoch_end)

        data=r"C:/Users/Amirah/OneDrive/999 PERSONAL/MT/data/data.yaml"  
        # Train with MLflow automatic logging
        results = model.train(
            data=data,
            epochs=1,
            imgsz=640,
            batch=8,
            name=mlflow_config['tracking']['run_name'],
            device=0,  # Use GPU if available
            save=True,
            classes=[0]
        )
        dataset =  r"C:/Users/Amirah/OneDrive/999 PERSONAL/MT/data/test" 
        # Validate and log metrics
        metrics = model.val(dataset)
        mlflow.log_metrics({
            'test_mAP50': metrics.box.map50,
            'test_mAP50-95': metrics.box.map,
            'test_precision': metrics.box.mp,
            'test_recall': metrics.box.mr
        })

        

        # Save and log model
        best_model_path = os.path.join('runs', 'detect', mlflow_config['tracking']['run_name'], 'weights', 'best.pt')
        trained_model.save(best_model_path)

        if os.path.exists(best_model_path):
            mlflow.log_artifact(best_model_path)
        else:
            print(f"File not found: {best_model_path}. Ensure training completed successfully.")
    return model, results


if __name__ == "__main__":

    trained_model,results = train_model()

New https://pypi.org/project/ultralytics/8.3.105 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.104  Python-3.10.0 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolo11l.pt, data=C:/Users/Amirah/OneDrive/999 PERSONAL/MT/data/data.yaml, epochs=1, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=yolov11-glasses9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=[0], re

train: Scanning C:\Users\Amirah\OneDrive\999 PERSONAL\MT\data\train\labels.cache... 857 images, 0 backgrounds, 0 corrupt: 100%|██████████| 857/857 [00:00<?, ?it/s]
val: Scanning C:\Users\Amirah\OneDrive\999 PERSONAL\MT\data\valid\labels.cache... 121 images, 0 backgrounds, 0 corrupt: 100%|██████████| 121/121 [00:00<?, ?it/s]


Plotting labels to runs\detect\yolov11-glasses9\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
MLflow: logging run_id(23add6fde1ea42809a391899c6064fef) to http://localhost:5000
MLflow: disable with 'yolo settings mlflow=False'
in the on_train_start
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\yolov11-glasses9
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1      5.92G      1.797      2.407       1.91          0        640: 100%|██████████| 108/108 [00:33<00:00,  3.25it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  25%|██▌       | 2/8 [00:15<00:47,  7.98s/it]


🏃 View run handsome-steed-510 at: http://localhost:5000/#/experiments/533916911720758159/runs/23add6fde1ea42809a391899c6064fef
🧪 View experiment at: http://localhost:5000/#/experiments/533916911720758159


RuntimeError: DataLoader worker (pid(s) 64176, 28936, 59132, 56024) exited unexpectedly

In [24]:
import mlflow
print(mlflow.active_run())  # Check if there's an active run
print(mlflow.get_experiment_by_name('/Shared/Ultralytics'))

None
<Experiment: artifact_location='mlflow-artifacts:/994303789912803226', creation_time=1744176554463, experiment_id='994303789912803226', last_update_time=1744265188700, lifecycle_stage='deleted', name='/Shared/Ultralytics', tags={}>


In [16]:
#train with --save_weights
trained_model.save("runs\detect\yolov11-glasses4\weights\best.pt")

NameError: name 'trained_model' is not defined

In [ ]:
import mlflow
from ultralytics import YOLO
import re
from datetime import datetime

def setup_mlflow():
    """Initialize MLflow tracking"""
    mlflow.set_tracking_uri("file:./mlflow")  # Local storage
    mlflow.set_experiment("yolo-custom-callbacks")
    
def on_train_start(trainer):
    """Callback when training starts"""
    print('\nTraining started!')
    with mlflow.start_run(run_name=f"yolov8-{datetime.now().strftime('%Y%m%d-%H%M%S')}"):
        # Log hyperparameters
        mlflow.log_params({
            'model': trainer.args.get('model', 'custom'),
            'data': trainer.args.get('data', ''),
            'epochs': trainer.args.get('epochs', 0),
            'batch_size': trainer.args.get('batch', -1),
            'imgsz': trainer.args.get('imgsz', 640),
            'device': trainer.args.get('device', 'gpu')
        })
        # Store the run ID for later use
        trainer.mlflow_run_id = mlflow.active_run().info.run_id

def on_fit_epoch_end(trainer):
    """Callback at the end of each training epoch"""
    print(f'\nEpoch {trainer.epoch} completed')
    
    # Convert metrics to MLflow format
    metrics_dict = {f"{re.sub('[()]', '', k)}": float(v) 
                   for k, v in trainer.metrics.items()}
    
    # Log hardware metrics
    if hasattr(trainer, 'gpu_mem'):
        metrics_dict.update({
            'hardware/gpu_util': trainer.gpu_mem[0],
            'hardware/gpu_mem': trainer.gpu_mem[1]
        })
    
    # Log learning rate
    for i, lr in enumerate(trainer.lr):
        metrics_dict[f'learning_rate/group_{i}'] = lr
    
    # Log to MLflow using the stored run ID
    with mlflow.start_run(run_id=trainer.mlflow_run_id):
        mlflow.log_metrics(metrics=metrics_dict, step=trainer.epoch)

def on_train_end(trainer):
    """Callback when training completes"""
    print('\nTraining finished!')
    with mlflow.start_run(run_id=trainer.mlflow_run_id):
        # Log final artifacts
        mlflow.log_artifacts(trainer.save_dir, "training_artifacts")
        
        # Log validation results if available
        if hasattr(trainer, 'validator'):
            results = trainer.validator.results
            mlflow.log_metrics({
                'val/mAP50': results.box.map50,
                'val/mAP50-95': results.box.map,
                'val/precision': results.box.mp,
                'val/recall': results.box.mr
            })

if __name__ == "__main__":
    # Setup MLflow
    setup_mlflow()
    
    # Load model
    model = YOLO('yolov11l.pt')
    
    # Register callbacks
    model.add_callback("on_train_start", on_train_start)
    model.add_callback("on_fit_epoch_end", on_fit_epoch_end)
    model.add_callback("on_train_end", on_train_end)
    

    # Start training
    results = model.train(
        data='coco8.yaml',
        epochs=10,
        imgsz=640,
        batch=16,
        device='0'  # Use GPU if available
    )

In [6]:
from src.config.config import load_config

app_config = load_config('configs/app_config.yaml')
metrics = trained_model.val(data=app_config['testing']['data'], agnostic_nms=True,save_conf=True, save_txt=True, save=True)
conf_matrix = metrics.confusion_matrix.matrix

Ultralytics 8.3.104  Python-3.10.0 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)


val: Scanning C:\Users\Amirah\OneDrive\999 PERSONAL\MT\GlassesDetection-2\valid\labels.cache... 121 images, 0 backgrounds, 0 corrupt: 100%|██████████| 121/121 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:04<00:00,  3.31it/s]


                   all        121        158      0.975      0.956      0.985      0.735
Speed: 0.6ms preprocess, 9.0ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\yolov11-glasses73


In [4]:
import cv2
import numpy as np
import mlflow
from ultralytics import YOLO
from typing import Tuple, Optional
import logging
from datetime import datetime
import os

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class GlassesDetectionPipeline:
    def __init__(self, model_path: str = 'yolov8n.pt'):
        """
        Initialize the glasses detection pipeline with YOLO model.
        
        Args:
            model_path: Path to YOLO model weights file
        """
        self.model = self._load_model(model_path)
        self.class_names = self.model.names
        self.glasses_class_id = self._get_glasses_class_id()
        
        # MLflow setup
        mlflow.set_tracking_uri("http://localhost:5000")  # Update if your MLflow server is elsewhere
        mlflow.set_experiment("Glasses Detection")
        
    def _load_model(self, model_path: str):
        """Load YOLO model with error handling."""
        try:
            model = YOLO(model_path)
            logger.info(f"Successfully loaded model from {model_path}")
            return model
        except Exception as e:
            logger.error(f"Failed to load model: {e}")
            raise
            
    def _get_glasses_class_id(self) -> Optional[int]:
        """Find the class ID for glasses in YOLO model."""
        for class_id, class_name in self.class_names.items():
            if 'glasses' in class_name.lower():
                return class_id
        logger.warning("Glasses class not found in model. Using face detection instead.")
        return None
    
    def detect_glasses(self, frame: np.ndarray) -> Tuple[np.ndarray, dict]:
        """
        Detect glasses in a frame using YOLO.
        
        Args:
            frame: Input image frame (BGR format)
            
        Returns:
            Tuple of (annotated_frame, detection_metrics)
        """
        # Convert BGR to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Run inference
        results = self.model(rgb_frame, verbose=False)
        
        # Process results
        annotated_frame = frame.copy()
        detection_metrics = {
            'num_detections': 0,
            'glasses_detected': False,
            'confidence': 0.0
        }
        
        for result in results:
            boxes = result.boxes
            for box in boxes:
                # Get box coordinates
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                
                # Check if this is a glasses detection
                if box.cls == self.glasses_class_id:
                    detection_metrics['num_detections'] += 1
                    detection_metrics['glasses_detected'] = True
                    detection_metrics['confidence'] = max(detection_metrics['confidence'], box.conf.item())
                    
                    # Replace glasses with black pixels
                    annotated_frame[y1:y2, x1:x2] = 0
                    
                    # Draw rectangle for visualization (optional)
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(annotated_frame, f"Glasses: {box.conf:.2f}", 
                                (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 
                                0.9, (0, 255, 0), 2)
        
        return annotated_frame, detection_metrics
    
    def process_video_stream(self, source: int = 0, output_dir: str = 'output'):
        """
        Process video stream from camera and apply glasses detection.
        
        Args:
            source: Camera source (usually 0 for default camera)
            output_dir: Directory to save output frames
        """
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        
        # Start video capture
        cap = cv2.VideoCapture(source)
        if not cap.isOpened():
            logger.error("Cannot open camera")
            return
            
        # Set camera resolution to 1080p
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
        
        # Start MLflow run
        with mlflow.start_run():
            logger.info("Starting MLflow run for glasses detection")
            
            try:
                frame_count = 0
                while True:
                    # Capture frame-by-frame
                    ret, frame = cap.read()
                    if not ret:
                        logger.error("Can't receive frame. Exiting...")
                        break
                        
                    # Process frame
                    processed_frame, metrics = self.detect_glasses(frame)
                    
                    # Log metrics to MLflow
                    mlflow.log_metrics({
                        'frame_num': frame_count,
                        'glasses_detected': int(metrics['glasses_detected']),
                        'confidence': metrics['confidence'],
                        'num_detections': metrics['num_detections']
                    }, step=frame_count)
                    
                    # Display the resulting frame
                    cv2.imshow('Glasses Detection', processed_frame)
                    
                    # Save frame every 30 frames (1 second at 30fps)
                    if frame_count % 30 == 0:
                        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
                        output_path = os.path.join(output_dir, f"frame_{timestamp}.jpg")
                        cv2.imwrite(output_path, processed_frame)
                        mlflow.log_artifact(output_path)
                    
                    frame_count += 1
                    
                    # Break the loop on 'q' key
                    if cv2.waitKey(1) == ord('q'):
                        break
                        
            finally:
                # When everything done, release the capture
                cap.release()
                cv2.destroyAllWindows()
                logger.info("Video processing completed")
                
            # Log final parameters
            mlflow.log_params({
                'model': 'YOLOv8',
                'task': 'glasses_detection',
                'resolution': '1080p',
                'glasses_class_id': self.glasses_class_id
            })

def main():
    """Main function to execute the pipeline."""
    try:
        logger.info("Starting Glasses Detection Pipeline")
        
        # Initialize pipeline
        pipeline = GlassesDetectionPipeline()
        
        # Process video stream
        pipeline.process_video_stream()
        
    except Exception as e:
        logger.error(f"Error in main execution: {e}")
        raise

if __name__ == "__main__":
    main()

INFO:__main__:Starting Glasses Detection Pipeline


KeyboardInterrupt: 